In [3]:
import os
import glob

import pandas as pd
import numpy as np

In [4]:
#1, 1
with open("/cluster/projects/radiomics/Temp/sejin/slurm_logs/1553757-aaai_predict.out", 'r') as f:
    out = f.readlines()

#1, 1
with open("/cluster/projects/radiomics/Temp/sejin/slurm_logs/1555461-aaai_predict.out", 'r') as f:
    out.extend(f.readlines())

#1, 2    
with open("/cluster/projects/radiomics/Temp/sejin/slurm_logs/1555474-aaai_predict.out", 'r') as f:
    out.extend(f.readlines())

In [80]:
pred_idx = []
for n, line in enumerate(out):
    if 'now testing' in line:
        pred_idx.append(n)

timestamp_dict = {}
for i in pred_idx:
    if "FAILED" not in out[i+1]:
        model_name = out[i].split("/")[0][-17:]
        timestamp  = out[i+97].split("]")[-1]
        if len(timestamp) < 14:
            timestamp  = out[i+96].split("]")[-1]
        if len(timestamp) < 14:
            timestamp  = out[i+129].split("]")[-1]
        timestamp_dict[timestamp.strip()] = model_name
        

In [11]:
results = pd.read_csv("./data/predictions/test_results_with_names.csv", index_col=-1)

In [12]:
def best_ckpt (arr):
    """
    Returns checkpoint with highest ROC_AUC or highest LOSS
    """
    best_idx, best_epoch, best_loss, best_auc = 0, 0, 0, 0
    
    for idx, c in enumerate(arr):
        epoch, loss, auc = c.split("/")[-1].split("-")
        epoch = float(epoch.split("=")[-1])
        loss  = float(loss.split("=")[-1])
        auc   = float(auc.split("=")[-1][:-5])
        if auc > best_auc:
            best_idx, best_epoch, best_loss, best_auc = idx, epoch, loss, auc
        elif auc == best_auc:
            if loss < best_loss:
                best_idx, best_epoch, best_loss, best_auc = idx, epoch, loss, auc
            elif loss == best_loss:
                if epoch > best_epoch:
                    best_idx, best_epoch, best_loss, best_auc = idx, epoch, loss, auc

    return arr[best_idx]

In [79]:
# just looking at list of best models (models used for inference)

models = sorted(glob.glob("/cluster/projects/radiomics/Temp/sejin/aaai21_survival_prediction/data/logs/aaai/dual_mtlr_155*"))

best_checkpoints = []
for model in models:
    ckpts = glob.glob(model+"/checkpoints/*")       
    if len(ckpts) > 0:
        print(model.split("/")[-1], best_ckpt(ckpts).split("/")[-1])


dual_mtlr_1550685 aaai_epoch=68-loss=2.01e+00-roc_auc=0.79.ckpt
dual_mtlr_1550686 aaai_epoch=68-loss=2.01e+00-roc_auc=0.79.ckpt
dual_mtlr_1550687 aaai_epoch=68-loss=2.01e+00-roc_auc=0.79.ckpt
dual_mtlr_1550688 aaai_epoch=68-loss=2.02e+00-roc_auc=0.79.ckpt
dual_mtlr_1550689 aaai_epoch=147-loss=2.01e+00-roc_auc=0.79.ckpt
dual_mtlr_1550691 aaai_epoch=121-loss=2.14e+00-roc_auc=0.78.ckpt
dual_mtlr_1550692 aaai_epoch=48-loss=2.04e+00-roc_auc=0.82.ckpt
dual_mtlr_1550693 aaai_epoch=200-loss=1.92e+00-roc_auc=0.81.ckpt
dual_mtlr_1550694 aaai_epoch=57-loss=2.02e+00-roc_auc=0.80.ckpt
dual_mtlr_1550695 aaai_epoch=61-loss=2.08e+00-roc_auc=0.83.ckpt
dual_mtlr_1550696 aaai_epoch=64-loss=2.28e+00-roc_auc=0.78.ckpt
dual_mtlr_1550697 aaai_epoch=191-loss=2.22e+00-roc_auc=0.79.ckpt
dual_mtlr_1553555 aaai_epoch=130-loss=1.98e+00-roc_auc=0.81.ckpt
dual_mtlr_1553710 aaai_epoch=75-loss=1.97e+00-roc_auc=0.81.ckpt
dual_mtlr_1553711 aaai_epoch=66-loss=2.00e+00-roc_auc=0.82.ckpt
dual_mtlr_1553712 aaai_epoch=36-los

In [96]:
one_layers  = ["dual_mtlr_1550693", "dual_mtlr_1550694", "dual_mtlr_1550695", 
               "dual_mtlr_1550696", "dual_mtlr_1550697", "dual_mtlr_1553722", 
               "dual_mtlr_1553725", "dual_mtlr_1553726", "dual_mtlr_1553727"]

two_layers  = ["dual_mtlr_1553555", "dual_mtlr_1553710", "dual_mtlr_1553711", 
               "dual_mtlr_1553713", "dual_mtlr_1553714", "dual_mtlr_1553715", 
               "dual_mtlr_1553716", "dual_mtlr_1553718", "dual_mtlr_1553719"]

In [97]:
def make_data(path, split="training"):
        """Load and preprocess the data."""
        try:
            df = pd.read_csv(path)
        except:
            df = path
            
        clinical_data = (df
                         .query("split == @split")
                         #.set_index("Study ID")
                         .drop(["split"], axis=1, errors="ignore"))
#         if split == "training":
        clinical_data = clinical_data.rename(columns={"death": "event", "survival_time": "time"})
        # Convert time to months
        clinical_data["time"] *= 12

        # binarize T stage as T1/2 = 0, T3/4 = 1
        clinical_data["T Stage"] = clinical_data["T Stage"].map(
            lambda x: "T1/2" if x in ["T1", "T1a", "T1b", "T2"] else "T3/4")

        # use more fine-grained grouping for N stage
        clinical_data["N Stage"] = clinical_data["N Stage"].map({
                                                                "N0":  "N0",
                                                                "N1":  "N1",
                                                                "N2":  "N2",
                                                                "N2a": "N2",
                                                                "N2b": "N2",
                                                                "N2c": "N2",
                                                                "N3":  "N3",
                                                                "N3a": "N3",
                                                                "N3b": "N3"})
        
        clinical_data["Stage"] = clinical_data["Stage"].map(
            lambda x: "I/II" if x in ["I", "II", "IIA"] else "III/IV")

        clinical_data["ECOG"] = clinical_data["ECOG"].map(
            lambda x: ">0" if x > 0 else "0")

        clinical_data = pd.get_dummies(clinical_data,
                                       columns=["Sex",
                                                "T Stage",
                                                "N Stage",
                                                "Disease Site",
                                                "Stage",
                                                "ECOG"],
                                       drop_first=True)
        
        clinical_data = pd.get_dummies(clinical_data, columns=["HPV Combined"])
        return clinical_data

# ensemble predictions

In [98]:
df1 = pd.read_csv("/cluster/projects/radiomics/RADCURE-challenge/clinical_cancer_death.csv")
df2 = pd.read_csv("/cluster/projects/radiomics/RADCURE-challenge/data/clinical_test.csv")
clinical_data = make_data(pd.merge(df1, df2[["Study ID", "ECOG"]], how='inner', on='Study ID'), split="test").drop("EGFRI", axis=1)

In [99]:
true_binary = clinical_data["target_binary"]
true_time   = clinical_data["time"]
true_event  = clinical_data["event"]
true_cancer = clinical_data["cancer_death"]

# CI + IBS

In [111]:
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss
from lifelines.utils import concordance_index

In [113]:
df_one_risk_event  = pd.DataFrame(index = clinical_data["Study ID"])
df_one_risk_cancer = pd.DataFrame(index = clinical_data["Study ID"])

for ts in timestamp_dict.keys():
    model_name  = timestamp_dict[ts]
    if model_name in one_layers:
        #print(model_name)
        risk_pred  = np.load(f"./data/predictions/{ts}_risk_pred.npy")
        df_one_risk_event[model_name]  = risk_pred[:,0]
        df_one_risk_cancer[model_name] = risk_pred[:,1]
        
        #risk_pred   = np.load(f"./data/predictions/{ts}_risk_pred.npy")


In [117]:
one_risk_event  = df_one_risk_event.mean(axis=1)
one_risk_cancer = df_one_risk_cancer.mean(axis=1)

ci_event   = concordance_index(true_time, -one_risk_event, event_observed=true_event)
ci_cancer  = concordance_index(true_time, -one_risk_cancer, event_observed=true_cancer)
#ibs_event  = brier_score_loss(true_event, one_risk_event)
#ibs_cancer = brier_score_loss(true_cancer, one_risk_cancer)

ci_event, ci_cancer#, ibs_event, ibs_cancer

Study ID
RADCURE300     0.929297
RADCURE301     1.805780
RADCURE315     2.199147
RADCURE320     1.975212
RADCURE323     2.488345
                 ...   
RADCURE4105    1.330033
RADCURE4108    1.976290
RADCURE4113    2.497861
RADCURE4116    3.058000
RADCURE4123    2.759536
Length: 750, dtype: float32 Study ID
RADCURE300     1.136167
RADCURE301     3.927481
RADCURE315     2.011226
RADCURE320     3.257865
RADCURE323     6.007787
                 ...   
RADCURE4105    1.781037
RADCURE4108    4.020158
RADCURE4113    6.475862
RADCURE4116    1.668721
RADCURE4123    0.734108
Length: 750, dtype: float32


(0.7264977397897948, 0.7759840710675447)

In [120]:
df_two_risk_event  = pd.DataFrame(index = clinical_data["Study ID"])
df_two_risk_cancer = pd.DataFrame(index = clinical_data["Study ID"])

for ts in timestamp_dict.keys():
    model_name  = timestamp_dict[ts]
    if model_name in two_layers:
        #print(model_name)
        risk_pred  = np.load(f"./data/predictions/{ts}_risk_pred.npy")
        df_two_risk_event[model_name]  = risk_pred[:,0]
        df_two_risk_cancer[model_name] = risk_pred[:,1]
        
        #risk_pred   = np.load(f"./data/predictions/{ts}_risk_pred.npy")


In [121]:
two_risk_event  = df_two_risk_event.mean(axis=1)
two_risk_cancer = df_two_risk_cancer.mean(axis=1)

ci_event   = concordance_index(true_time, -two_risk_event, event_observed=true_event)
ci_cancer  = concordance_index(true_time, -two_risk_cancer, event_observed=true_cancer)
#ibs_event  = brier_score_loss(true_event, one_risk_event)
#ibs_cancer = brier_score_loss(true_cancer, one_risk_cancer)

ci_event, ci_cancer#, ibs_event, ibs_cancer

(0.7484492602438559, 0.7997855720631031)

# ROC_AUC (+ precision)

## one dense

In [102]:
df_one_event  = pd.DataFrame(index = clinical_data["Study ID"])
df_one_cancer = pd.DataFrame(index = clinical_data["Study ID"])

for ts in timestamp_dict.keys():
    model_name  = timestamp_dict[ts]
    if model_name in one_layers:
        print(model_name)
        event_pred  = pd.read_csv(f"./data/predictions/{ts}_event_pred.csv", index_col=0, header=None)
        cancer_pred = pd.read_csv(f"./data/predictions/{ts}_cancer_pred.csv", index_col=0, header=None)
        
        df_one_event[model_name]  = event_pred[1]
        df_one_cancer[model_name] = cancer_pred[1]
        
        #risk_pred   = np.load(f"./data/predictions/{ts}_risk_pred.npy")


dual_mtlr_1550693
dual_mtlr_1550694
dual_mtlr_1550695
dual_mtlr_1550696
dual_mtlr_1550697
dual_mtlr_1553722
dual_mtlr_1553725
dual_mtlr_1553726
dual_mtlr_1553727


In [103]:
one_pred_event  = df_one_event.mean(axis=1).tolist()
one_pred_cancer = df_one_cancer.mean(axis=1).tolist()

roc_auc_event   = roc_auc_score(true_event, one_pred_event)
roc_auc_cancer  = roc_auc_score(true_cancer, one_pred_cancer)
avg_prec_event  = average_precision_score(true_event, one_pred_event)
avg_prec_cancer = average_precision_score(true_cancer, one_pred_cancer)

roc_auc_event, roc_auc_cancer, avg_prec_event, avg_prec_cancer

(0.755674341147562,
 0.7642217380779472,
 0.44114351290922404,
 0.3282840802479536)

In [104]:
one_pred_event  = df_one_event.median(axis=1).tolist()
one_pred_cancer = df_one_cancer.median(axis=1).tolist()

roc_auc_event   = roc_auc_score(true_event, one_pred_event)
roc_auc_cancer  = roc_auc_score(true_cancer, one_pred_cancer)
avg_prec_event  = average_precision_score(true_event, one_pred_event)
avg_prec_cancer = average_precision_score(true_cancer, one_pred_cancer)

roc_auc_event, roc_auc_cancer, avg_prec_event, avg_prec_cancer

(0.7553016804667574,
 0.7602805010893245,
 0.44780834120199253,
 0.3215777428060958)

## two dense

In [105]:
df_two_event  = pd.DataFrame(index = clinical_data["Study ID"])
df_two_cancer = pd.DataFrame(index = clinical_data["Study ID"])

for ts in timestamp_dict.keys():
    model_name  = timestamp_dict[ts]
    if model_name in two_layers:
        #print(model_name)
        event_pred  = pd.read_csv(f"./data/predictions/{ts}_event_pred.csv", index_col=0, header=None)
        cancer_pred = pd.read_csv(f"./data/predictions/{ts}_cancer_pred.csv", index_col=0, header=None)
        
        df_two_event[model_name]  = event_pred[1]
        df_two_cancer[model_name] = cancer_pred[1]
        
        #risk_pred   = np.load(f"./data/predictions/{ts}_risk_pred.npy")


In [106]:
two_pred_event  = df_two_event.mean(axis=1).tolist()
two_pred_cancer = df_two_cancer.mean(axis=1).tolist()

roc_auc_event2   = roc_auc_score(true_event, two_pred_event)
roc_auc_cancer2  = roc_auc_score(true_cancer, two_pred_cancer)
avg_prec_event2  = average_precision_score(true_event, two_pred_event)
avg_prec_cancer2 = average_precision_score(true_cancer, two_pred_cancer)

roc_auc_event2, roc_auc_cancer2, avg_prec_event2, avg_prec_cancer2

(0.7769392912459678,
 0.7883230452674896,
 0.46042158857443766,
 0.35532258331813427)

In [107]:
two_pred_event  = df_two_event.median(axis=1).tolist()
two_pred_cancer = df_two_cancer.median(axis=1).tolist()

roc_auc_event2   = roc_auc_score(true_event, two_pred_event)
roc_auc_cancer2  = roc_auc_score(true_cancer, two_pred_cancer)
avg_prec_event2  = average_precision_score(true_event, two_pred_event)
avg_prec_cancer2 = average_precision_score(true_cancer, two_pred_cancer)

roc_auc_event2, roc_auc_cancer2, avg_prec_event2, avg_prec_cancer2

(0.7759959938976813,
 0.7865982812878236,
 0.45395347199266994,
 0.34486025431219147)